<a href="https://colab.research.google.com/github/Jegovila/Taller-Vision/blob/main/1_CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D

from sklearn import metrics
import cv2

!wget -nc "https://raw.githubusercontent.com/Jegovila/Taller-Vision/main/datasetCNN/tres.png"

In [ ]:
(xtrain, ytrain), (xtest, ytest) = mnist.load_data()

In [ ]:
print(ytrain)

In [ ]:
number = np.random.randint(0,60000);

plt.imshow(xtrain[number],cmap=plt.cm.gray)
plt.title('numer = ' + str(ytrain[number]))
plt.show()

In [ ]:
xtrain = xtrain.astype('float32')/255
xtest = xtest.astype('float32')/255

xtrain = xtrain.reshape(xtrain.shape[0], 28, 28, 1)
xtest = xtest.reshape(xtest.shape[0], 28, 28, 1)

In [ ]:
ytrain = keras.utils.to_categorical(ytrain, 10)
ytest = keras.utils.to_categorical(ytest, 10)

In [ ]:
model = Sequential()

# Añadimos la primera capa
model.add(Conv2D(25,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))

# Añadimos la segunda capa
model.add(Conv2D(25,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Hacemos un flatten para poder usar una red fully connected
model.add(Flatten())
model.add(Dense(50,activation='relu'))
model.add(Dense(25,activation='relu'))

# Añadimos una capa softmax para que podamos clasificar las imágenes
model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# loss <- funcion de perdida
# keras.losses.categorical_crossentropy <- clasificacion multiclase
# keras.losses.binary_crossentropy <- clasificacion binaria
# keras.losses.mean_squared_error <- regresion

In [ ]:
hist = model.fit(xtrain,ytrain,batch_size=64,epochs=6,verbose=1,validation_data=(xtest,ytest))

# loss, accuracy <- para entrenamiento
# val_loss, val_accuracy <- para validacion

In [ ]:
# summarize history for accuracy
plt.figure()
plt.grid()
plt.plot(hist.history['accuracy'],lw=2)
plt.plot(hist.history['val_accuracy'],lw=2)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.figure()
plt.grid()
plt.plot(hist.history['loss'],lw=2)
plt.plot(hist.history['val_loss'],lw=2)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
ypred = model.predict(xtest)

y_pred = np.argmax(ypred,axis=1)
y_test = np.argmax(ytest,axis=1)

# Métricas
print('Metricas: \n', metrics.classification_report(y_test,y_pred))

# Matriz de Confusión
print('Confusion matrix: \n', metrics.confusion_matrix(y_test,y_pred))

In [ ]:
model.save('CNN_Model_MNIST.h5')

In [ ]:
model = keras.models.load_model('CNN_Model_MNIST.h5')

In [ ]:
patron = xtest[np.random.randint(0,10000)].reshape(1,28,28,1)

number = np.argmax(model.predict(patron),axis=1);

plt.imshow(patron[0,:,:,0],cmap=plt.cm.gray)
plt.title('Prediction: ' + str(number))
plt.show()

In [ ]:
img = cv2.imread('tres.png')
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

th, img_bn = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV)
img_bn = cv2.resize(img_bn, (28,28))

patron = img_bn.reshape(1,28,28,1)/255

predection = model.predict(patron);
number = np.argmax(model.predict(patron),axis=1);

plt.imshow(patron[0,:,:,0],cmap=plt.cm.gray)
plt.title('Prediction: ' + str(number))
plt.show()